# Decision Tree

TODO:
- brief intro
- categorial vs numerical features
- continuous vs discrete data
- bagging
- gini/cross entropy/...

## Covertype dataset (link)

TODO: brief intro

In [ ]:
# Import packages
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split

In [ ]:
# Load covertype dataset
data = fetch_covtype()

# Extract features and target variable
X = data.data  # features
y = data.target  # target labels

# Split the dataset into 80% train set and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Workshop 6 review task:

(Optional: If you didn't come to workshop 6 or you don't know what is linear regression, you can skip this exercise)

TODO: 

use linear regression to train model by using the train set, and then test accuracy on the test set

Think: how would you preprocess the data?

In [ ]:
# TODO: write your code here

## Decision Tree for classification

TODO: use a decision tree to predict which covertype does the given data belong to

TODO: use xx method in sklearn (link) 

intro each param, what does it mean? how would it affect the rlt?

In [ ]:
# TODO: code

(add a lightbulb emoji before each "Think") Think: compared to linear regression, which has a higher accuracy in this prediction task? What causes the differences?

## How do we choose a model
TODO:
- validation set (divide from the train set, 20% for example)
- gird search (need intro of grid search)
- Bayesian search
- ...

## Using Grid search to choose the "best" model

In [ ]:
# TODO: code

In [ ]:
# TODO: test the accuracy on the test set

# Random Forest

TODO: intro

TODO: method, params in the method (what does it mean? how would it affect the rlt?)

In [ ]:
# TODO: code

Think: compared to a single decision tree, which has a higher accuracy in this prediction task? What do you think is the reason behind this? Would it always hold in all tasks and situations?

# Optional Task
 
Compared test accuracy of decision trees by using preprocessed data and unpreprocessed(is this a word?) data, how is the results? Why?

In [ ]:
# TODO: write your codes here